# Configuração do ambiente

## Import das bibliotecas

In [0]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from typing import Optional

## Definição dos links e diretórios

In [0]:
# Captura o diretório raiz do projeto
dir_projeto = Path(os.getcwd()).parent.resolve()

In [0]:
# Mapeia o diretório que está no Bucket S3 com os arquivos do projeto
dir_s3_dados = r's3a://databricks-workspace-stack-edf2c-bucket/unity-catalog/3805457818561400/case-tecnico-analise-dados-ifood/dados'
dir_s3_dados_brutos = f'{dir_s3_dados}/brutos'
dir_s3_dados_processados = f'{dir_s3_dados}/processados'


## Configurações do Plotly

In [0]:
# Altera o tema e o formato de renderização dos gráficos do Plotly
pio.templates.default = 'plotly_white'
pio.renderers.default = 'vscode+pdf+png+jpeg'

In [0]:
# Função para plotar histogramas sobrepostos para comparação entre grupos
def histogramas_sobrepostos(
    df: pd.DataFrame,
    variavel_numerica: str,
    variavel_binaria: str,
    coluna_frequencia: str,
    valor_positivo: str,
    valor_negativo: str,
    titulo: Optional[str] = None,
    subtitulo: Optional[str] = None,
    titulo_eixo_x: Optional[str] = None,
    titulo_eixo_y: Optional[str] = None,
    nbinsx: int = 30
) -> go.Figure:
    """
    Gera histogramas sobrepostos com base em uma variável numérica e uma coluna de frequência,
    segmentando os dados por uma variável binária.

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame contendo as colunas de interesse.
    variavel_numerica : str
        Nome da variável numérica a ser plotada no eixo X.
    variavel_binaria : str
        Nome da variável categórica binária usada para segmentação.
    coluna_frequencia : str
        Nome da coluna que representa a frequência de cada linha.
    valor_positivo : str
        Valor considerado como "1" na variável binária (ex: "target").
    valor_negativo : str
        Valor considerado como "0" na variável binária (ex: "control").
    titulo : str, opcional
        Título do gráfico.
    subtitulo : str, opcional
        Subtítulo do gráfico.
    titulo_eixo_x : str, opcional
        Título do eixo X.
    titulo_eixo_y : str, opcional
        Título do eixo Y.
    nbinsx : int, opcional
        Número de bins no eixo X (padrão 30).

    Retorno
    -------
    go.Figure
        Gráfico com histogramas sobrepostos.
    """

    # Separa os dados em dois grupos, com base na variável binária
    grupo_positivo = df[df[variavel_binaria] == valor_positivo]
    grupo_negativo = df[df[variavel_binaria] == valor_negativo]

    # Cria a figura e adiciona os dois histogramas ponderados
    fig = go.Figure()

    fig.add_trace(
        go.Histogram(
            x=grupo_negativo[variavel_numerica],
            y=grupo_negativo[coluna_frequencia],
            name=valor_negativo,
            nbinsx=nbinsx,
            histfunc="sum",
            histnorm="probability",
            marker=dict(color='#000000')
        )
    )

    fig.add_trace(
        go.Histogram(
            x=grupo_positivo[variavel_numerica],
            y=grupo_positivo[coluna_frequencia],
            name=valor_positivo,
            nbinsx=nbinsx,
            histfunc="sum",
            histnorm="probability",
            marker=dict(color='#EA1D2C')
        )
    )

    # Layout e estilo
    fig.update_layout(
        barmode='overlay',
        width=800,
        height=550,
        title=titulo,
        # title_subtitle_text=subtitulo,
        xaxis_title=titulo_eixo_x,
        yaxis_title=titulo_eixo_y,
        legend=dict(orientation="h", yanchor="bottom", y=0.99, xanchor="center", x=0.5),
        showlegend=True
    )

    fig.update_traces(opacity=0.65)
    fig.update_xaxes(tickformat=',', showgrid=False)
    fig.update_yaxes(tickformat=',.0%')

    return fig


# Dados

## Download

### Pedidos

In [0]:
# Lê o arquivo .parquet do diretório de dados processados
pedidos = spark.read.parquet(f"{dir_s3_dados_processados}/pedidos")

display(pedidos.limit(100))

#### Pedidos por estado

In [0]:
display(
    pedidos.groupBy("origin_platform") \
    .agg(count("order_id").alias("quantidade_pedidos")) \
    .orderBy("quantidade_pedidos", ascending=False) \
)

#### Pedidos por plataforma

In [0]:
display(
    pedidos.groupBy("order_scheduled") \
    .agg(count("order_id").alias("quantidade_pedidos")) \
    .orderBy("quantidade_pedidos", ascending=False) \
)

#### Pedidos agendados

In [0]:
display(
    pedidos.groupBy("delivery_address_state") \
    .agg(count("order_id").alias("quantidade_pedidos")) \
    .orderBy("quantidade_pedidos", ascending=False) \
)

### Usuários

In [0]:
# Lê o arquivo .parquet do diretório de dados processados
usuarios = spark.read.parquet(f"{dir_s3_dados_processados}/usuarios")

display(usuarios.limit(100))

#### Idiomas dos usuários

In [0]:
display(
    usuarios
    .groupBy("language") \
    .agg(count("customer_id") \
    .alias("quantidade_usuarios")) \
    .orderBy("quantidade_usuarios", ascending=False) \
)


### Restaurantes

In [0]:
# Lê o arquivo .parquet do diretório de dados processados
restaurantes = spark.read.parquet(f"{dir_s3_dados_processados}/restaurantes")

display(restaurantes.limit(100))

### Usuários teste A/B

In [0]:
# Lê o arquivo .parquet do diretório de dados processados
usuarios_teste_ab = spark.read.parquet(f"{dir_s3_dados_processados}/usuarios_teste_ab")

display(usuarios_teste_ab.limit(100))

## Registro de views temporárias

In [0]:
# Registro de views para uso posterior no spark SQL
pedidos.createOrReplaceTempView("vw_pedidos")
usuarios.createOrReplaceTempView("vw_usuarios")
restaurantes.createOrReplaceTempView("vw_restaurantes")
usuarios_teste_ab.createOrReplaceTempView("vw_usuarios_teste_ab")

-----

# Análise

1. No iFood, várias áreas utilizam testes A/B para avaliar o impacto de ações em diferentes métricas. Esses testes permitem validar hipóteses de crescimento e a viabilidade de novas funcionalidades em um grupo restrito de usuários. Nos dados fornecidos nesse case você encontrará uma marcação de usuários, separando-os entre grupo teste e controle de uma **campanha de cupons, que disponibilizou para os usuários do grupo teste um cupom especial**.

    a) **Defina os indicadores** relevantes para mensurar o sucesso da campanha e analise se ela teve impacto significativo dentro do período avaliado.

    b) Faça uma **análise de viabilidade financeira** dessa iniciativa como alavanca de crescimento, adotando as premissas que julgar necessárias (explicite as premissas adotadas). 

    c) **Recomende oportunidades de melhoria** nessa ação e **desenhe uma nova proposta de teste A/B** para validar essas hipóteses.

## a) **Defina os indicadores** relevantes para mensurar o sucesso da campanha e analise se ela teve impacto significativo dentro do período avaliado.

### Retenção de clientes

Quantos % dos usuários fizeram um novo pedido após o primeiro pedido com o cupom?

In [0]:
# Verificação do período dos dados de pedidos
datas = pedidos.agg(
    min("order_created_at").alias("data_inicio"),
    max("order_created_at").alias("data_fim")
).first()

data_inicio_pedidos = datas["data_inicio"]
data_fim_pedidos = datas["data_fim"]

print(f"Data de início dos pedidos: {data_inicio_pedidos}")
print(f"Data de fim dos pedidos: {data_fim_pedidos}")

In [0]:
# Verificação do período do cadastro de novos usuários
datas = usuarios.agg(
    min("created_at").alias("data_inicio"),
    max("created_at").alias("data_fim")
).first()

data_inicio_usuarios = datas["data_inicio"]
data_fim_usuarios = datas["data_fim"]

print(f"Data de início de cadastro de novos usuários: {data_inicio_usuarios}")
print(f"Data de fim de cadastro de novos usuários: {data_fim_usuarios}")

Tanto os usuários do grupo de tratamento quanto do grupo controle tiveram seu cadastro feito há mais de 6 meses antes do início da ação, então não é possível analizar uma métrica de 'atração de novos clientes' e também não é necessário se preocupar com intervalos de tempo diferentes de participação na campanha (todos participaram os 2 meses).

In [0]:
query_retencao = """
WITH usuarios_classificados AS (
    SELECT
        p.customer_id,
        ab.is_target,
        COUNT(DISTINCT p.order_id) AS quantidade_pedidos,
        CASE
            WHEN COUNT(DISTINCT p.order_id) > 1 THEN 1
            ELSE 0
        END AS retido
    FROM
        vw_pedidos AS p
    JOIN
        vw_usuarios_teste_ab AS ab ON
            p.customer_id = ab.customer_id
    GROUP BY
        p.customer_id,
        ab.is_target
)

SELECT
    is_target,
    retido,
    quantidade_pedidos,
    COUNT(*) AS total_usuarios
FROM
    usuarios_classificados
GROUP BY
    is_target,
    retido,
    quantidade_pedidos
"""

info_retencao = spark.sql(query_retencao).toPandas()

info_retencao.head()

In [0]:
# Quantos % dos usuários foram retidos em cada um dos grupos do teste?
# Agrupa por grupo do teste e se foi retido ou não
resumo_retencao = info_retencao.groupby(['is_target', 'retido'])['total_usuarios'].sum().unstack(fill_value=0)

# Renomeia colunas
resumo_retencao.rename(columns = {0: 'usuarios_nao_retidos', 1: 'usuarios_retidos'}, inplace=True)

# Calcula totais e porcentagem
resumo_retencao['usuarios_totais'] = resumo_retencao['usuarios_retidos'] + resumo_retencao['usuarios_nao_retidos']
resumo_retencao['porcentagem_retencao'] = (resumo_retencao['usuarios_retidos'] / resumo_retencao['usuarios_totais']) * 100

# Formata como tabela final
resumo_retencao = resumo_retencao.reset_index()

display(resumo_retencao)

Vemos que os usuários do **grupo target possuem 10 p.p. a mais de retenção** do que o grupo controle.

Enquanto o primeiro grupo apresentam uma taxa de retenção de 57,6%, o grupo controle possui uma taxa de 47,6%.

### Média de pedidos por usuário

Nesse período, os usuários fizeram quantos pedidos em média na plataforma?

In [0]:
# Calcula o total de pedidos e total de usuários por grupo
resumo_pedidos_medios = info_retencao.groupby("is_target").apply(
    lambda df: pd.Series({
        "total_pedidos": (df["quantidade_pedidos"] * df["total_usuarios"]).sum(),
        "total_usuarios": df["total_usuarios"].sum()
    })
)

# Calcula a média de pedidos por usuário
resumo_pedidos_medios["media_pedidos_por_usuario"] = resumo_pedidos_medios["total_pedidos"] / resumo_pedidos_medios["total_usuarios"]

resumo_pedidos_medios = resumo_pedidos_medios.reset_index()

display(resumo_pedidos_medios)

Analisando a métrica de média de pedidos por usuário dentro dessa janela de 2 meses é possível ver um desempenho melhor do grupo target (com 3,2 pedidos/pessoa) em comparação ao desempenho do grupo controle (2,8 pedidos/pessoa). Isso representa um aumento de 13,3%!

Mas como são distribuídos essa quantidade de pedidos por usuário entre os dois grupos do teste A/B?

In [0]:
# Verificação da proporção de usuários com muitos pedidos para não usar no histograma
limiar_pedidos = 20
usuarios_mais_pedidos = info_retencao[info_retencao['quantidade_pedidos'] > limiar_pedidos]['total_usuarios'].sum()

# Total geral de usuários
usuarios_total = info_retencao['total_usuarios'].sum()

# Porcentagem
porcentagem_mais_pedidos = (usuarios_mais_pedidos / usuarios_total) * 100

print(f"{porcentagem_mais_pedidos:.2f}% dos usuários fizeram mais de {limiar_pedidos} pedidos.")

Como a proproção de colaboradores que fizeram mais de 20 pedidos nesse período de 2 meses é menor do que 1%, eles serão desconsiderados dos dados usados para montar o histograma abaixo.

In [0]:
# Como estão distribuídos os usuários por grupo do teste e por quantidade de pedidos no período?
fig = histogramas_sobrepostos(
    df=info_retencao.query("quantidade_pedidos <= 20"), # Descarta os outliers para melhor visualização do histograma
    variavel_numerica="quantidade_pedidos",
    variavel_binaria="is_target",
    coluna_frequencia="total_usuarios",
    valor_positivo="target",
    valor_negativo="control",
    titulo="Distribuição de pedidos por grupo",
    titulo_eixo_x="Quantidade de pedidos",
    titulo_eixo_y="Proporção de usuários",
)

fig.show()

Apesar dos dois grupos terem distribuições assimétricas positivas, é possível perceber um deslocamento maior da curva de usuários do grupo target para a direita, confirmando a maior média de pedidos por usuário.

### Ticket médio por pedido e por usuário

Qual foi o gasto médio a cada pedido feito? E quanto foi gasto em média por cada usuário?

In [0]:
query_ticket_medio = """
WITH pedidos_com_info AS (
    SELECT
        p.customer_id,
        ab.is_target,
        p.order_id,
        p.order_total_amount
    FROM
        vw_pedidos AS p
    JOIN
        vw_usuarios_teste_ab AS ab ON
          p.customer_id = ab.customer_id
),

metricas_agrupadas AS (
    SELECT
        is_target,
        customer_id,
        COUNT(order_id) AS quantidade_pedidos,
        SUM(order_total_amount) AS valor_total
    FROM
        pedidos_com_info
    GROUP BY
        is_target,
        customer_id
)

SELECT
    is_target,
    SUM(valor_total) AS valor_total_pedidos,
    SUM(quantidade_pedidos) AS quantidade_total_pedidos,
    COUNT(*) AS quantidade_total_usuarios,
    ROUND(SUM(valor_total) / COUNT(*), 2) AS ticket_medio_por_usuario,
    ROUND(SUM(valor_total) / SUM(quantidade_pedidos), 2) AS ticket_medio_por_pedido
FROM
    metricas_agrupadas
GROUP BY
  is_target
"""

ticket_medio = spark.sql(query_ticket_medio).toPandas()

display(ticket_medio)

Analisando a métrica de ticket médio por usuário, o grupo target obteve o valor de R$ 228,76, enquanto o grupo controle obteve R$ 202,67, mostrando que em média há um faturamento maior com os usuários que receberam o cupom de desconto.

Já olhando o ticket médio por pedido, os usuários que não receberam o cupom gastaram em média R$ 47,90 a cada compra, R$ 0,16 a mais do que a média gasta por pedido do que o público que recebeu os cupons (R$ 47,74). Entretanto, é importante destacar que os valores foram muito próximos mesmo com o desconto oferecido pelo Ifood, indicando um consumo maior desse público e resultando em um LTV (valor de vida útil do cliente) potencialmente maior.

## b) Faça uma análise de viabilidade financeira dessa iniciativa como alavanca de crescimento, adotando as premissas que julgar necessárias (explicite as premissas adotadas).

### Premissas

Como não foram informados os valores dos cupons, em quais compras eles foram aplicados e nem quantos cupons foram distribuídos para cada usuário do grupo target, serão feitas algumas premissas para permitir a análise de viabilidade financeira:

- O **cupom deu um desconto de R$ 10,00** quando aplicado a alguma compra feita pelos usuários do grupo target;
- Cada usuário do grupo target **recebeu apenas 1 cupom** e ele foi usado na primeira compra feita nesse período analisado;
- A receita incremental trazida pela ação dos cupons pode ser calculado como:
  - receita_total_incremental = (ticket_medio_por_usuario_target - ticket_medio_por_usuario_controle) * quantidade_usuarios_target_com_alguma_compra
- O custo total da campanha pode ser calculado como R$ 10,00 vezes a quantidade de usuários target que fizeram pelo menos uma compra no período:
  - custo_total_campanha = 10 * quantidade_usuarios_target_com_alguma_compra
- O **ROI (Retorno sobre o investimento)** para a ação pode ser calculado como:
  - ROI = (receita_total_incremental - custo_total_campanha) / custo_total_campanha

In [0]:
# Parâmetros conhecidos
ticket_medio_usuario_target = ticket_medio.loc[ticket_medio.is_target == "target", "ticket_medio_por_usuario"].iloc[0]
ticket_medio_usuario_controle = ticket_medio.loc[ticket_medio.is_target == "control", "ticket_medio_por_usuario"].iloc[0]
quantidade_usuarios_target_com_compra = ticket_medio.loc[ticket_medio.is_target == "target", "quantidade_total_usuarios"].iloc[0]

# Cálculo da receita incremental total
receita_incremental = (ticket_medio_usuario_target - ticket_medio_usuario_controle) * quantidade_usuarios_target_com_compra

print(f"Receita total incremental estimada: R$ {receita_incremental:,.2f}".replace(".", "|").replace(",", ".").replace("|", ","))


In [0]:
valor_cupom = 10.00
# Assume-se que todos os usuários do grupo target que fizeram pelo menos uma compra usaram um cupom
custo_total_campanha = valor_cupom * quantidade_usuarios_target_com_compra

print(f"Custo total estimado da campanha: R$ {custo_total_campanha:,.2f}".replace(".", "|").replace(",", ".").replace("|", ","))


In [0]:
roi = (receita_incremental - custo_total_campanha) / custo_total_campanha

print(f"ROI estimado da campanha: {roi:.2%}".replace(".", ","))


Apesar do ROI estimado de 160,9% parecer elevado, ele se refere à receita incremental bruta atribuída à campanha, sem considerar custos operacionais adicionais. Ainda assim, o resultado sugere que a ação foi financeiramente vantajosa.

**Limitações da análise**

Esta análise foi conduzida com base em dados disponíveis apenas no período da campanha e não contou com informações históricas anteriores. Além disso, a avaliação de viabilidade financeira se baseou em premissas simplificadas que foram adotadas devido à ausência de algumas informações-chave.

As principais limitações incluem:

- **Valor real dos cupons não informado**: adotou-se uma estimativa de R$ 10,00 por cupom, aplicada uma única vez por usuário;
- **Frequência e critérios de uso dos cupons desconhecidos**: não há controle sobre em quais pedidos os cupons foram aplicados ou quantos cupons foram efetivamente utilizados por usuário;
- **Receita incremental estimada com base no ticket médio por usuário**: essa métrica não reflete diretamente o lucro gerado, pois não desconta custos de operação (ex: comissões de restaurante, logística, marketing, etc.);
- **Margem de contribuição não considerada**: a análise foca na receita bruta incremental, sem considerar a margem líquida ou retorno real para o negócio;
- **Sem histórico pré-campanha**: não foi possível comparar o comportamento dos usuários antes e depois da campanha;
- **Resultados limitados ao curto prazo**: não foi possível mensurar o efeito de longo prazo da ação sobre o ciclo de vida do cliente (LTV).

Apesar dessas limitações, os resultados sugerem que a campanha gerou um impacto positivo na retenção e no engajamento dos usuários, com um ROI estimado atrativo mesmo sob premissas conservadoras.


-----

2. A criação de segmentações permite agrupar usuários de acordo com características e comportamentos similares, possibilitando criar estratégias direcionadas de acordo com o perfil de cada público, facilitando a personalização e incentivando o engajamento, retenção, além de otimização de recursos. Segmentações de usuários são muito utilizadas pelos times de Data, mas a área em que você atua ainda não tem segmentos bem definidos e cada área de Negócio utiliza conceitos diferentes. Por isso, você precisa:

    a) **Definir as segmentações** que fazem sentido especificamente **para o teste A/B** que está analisando.

    b) Estabelecer **quais serão os critérios utilizados para cada segmento** sugerido no item a). Utilize os critérios/ferramentas que achar necessários, mas lembre-se de explicar o racional utilizado na criação.

    c) Analisar os **resultados do teste A/B com base nos segmentos** definidos nos itens a) e b).

## a) **Definir as segmentações** que fazem sentido especificamente **para o teste A/B** que está analisando.
## b) Estabelecer quais serão os critérios utilizados para cada segmento sugerido no item a). Utilize os critérios/ferramentas que achar necessários, mas lembre-se de explicar o racional utilizado na criação.

Considerando os dados disponibilizados para a análise dessa campanha de cupons, pode-se segmentar os clientes da seguinte forma:

- **Intensidade de uso da plataforma (quantidade de pedidos por usuário):**  
  Usuários com diferentes níveis de engajamento podem responder de forma distinta à oferta de cupons. Para essa análise, os usuários podem ser segmentados com base no número de pedidos realizados durante o período avaliado, utilizando os seguintes grupos:
  - **Baixo uso:** 1 pedido
  - **Uso moderado:** 2 a 5 pedidos
  - **Alto uso:** mais de 5 pedidos

- **Ticket médio por compra do usuário:**  
  Essa segmentação ajuda a identificar se o cupom teve maior impacto entre consumidores com diferentes padrões de gasto. Será calculado o ticket médio de cada usuário e eles serão separados em três faixas:
  - **Baixo ticket:** usuários no tercil inferior (até o 33º percentil)
  - **Médio ticket:** usuários entre o 34º e o 66º percentil
  - **Alto ticket:** usuários acima do 66º percentil

- **Estado (UF) do pedido:**  
  A geolocalização pode influenciar o comportamento de consumo, seja por variações regionais na renda, oferta de restaurantes, ou estratégias de marketing local. Agrupar os usuários por unidade federativa permite observar se tiveram diferenças relevantes no desempenho da campanha entre regiões.

- **Plataforma onde foi feito o pedido:**  
  Usuários de diferentes plataformas (Android, iOS, Windows Phone, Desktop) podem apresentar perfis e hábitos distintos. A segmentação com base na variável `origin_platform` permite avaliar se a campanha teve maior impacto em alguma plataforma específica.

Essas segmentações permitem não apenas analisar o efeito do cupom em diferentes perfis, mas também embasam decisões futuras sobre personalização e direcionamento de campanhas de incentivo.


## c) Analisar os **resultados do teste A/B com base nos segmentos** definidos nos itens a) e b).

### Intensidade de uso da plataforma

In [0]:
query_segmentacao_uso = """
WITH pedidos_por_usuario AS (
    SELECT
        p.customer_id,
        ab.is_target,
        COUNT(p.order_id) AS quantidade_pedidos,
        SUM(p.order_total_amount) AS valor_total_pedidos
    FROM
        vw_pedidos AS p
    JOIN
        vw_usuarios_teste_ab AS ab ON
            p.customer_id = ab.customer_id
    GROUP BY
        p.customer_id,
        ab.is_target
),

usuarios_com_categoria AS (
    SELECT
        *,
        CASE
            WHEN quantidade_pedidos = 1 THEN '1) baixo uso'
            WHEN quantidade_pedidos BETWEEN 2 AND 5 THEN '2) uso moderado'
            ELSE '3) alto uso'
        END AS categoria_uso
    FROM
        pedidos_por_usuario
),

agregado_por_categoria AS (
    SELECT
        categoria_uso,
        is_target,
        COUNT(*) AS quantidade_total_usuarios,
        SUM(quantidade_pedidos) AS quantidade_total_pedidos,
        ROUND(SUM(quantidade_pedidos) * 1.0 / COUNT(*), 2) AS pedidos_medios_por_usuario,
        ROUND(SUM(valor_total_pedidos) / COUNT(*), 2) AS ticket_medio_por_usuario,
        ROUND(SUM(valor_total_pedidos) / SUM(quantidade_pedidos), 2) AS ticket_medio_por_pedido
    FROM
        usuarios_com_categoria
    GROUP BY
        categoria_uso,
        is_target
)

SELECT
    *
FROM
    agregado_por_categoria
ORDER BY
    categoria_uso,
    is_target
"""

df_segmentacao_uso = spark.sql(query_segmentacao_uso)

display(df_segmentacao_uso)


In [0]:
# Converte o dataframe completo de uso para Pandas (caso ainda não tenha feito)
df_uso = df_segmentacao_uso.toPandas()

# Separa os dados de target e controle
df_target = df_uso[df_uso['is_target'] == 'target'].copy()
df_control = df_uso[df_uso['is_target'] == 'control'][['categoria_uso', 'ticket_medio_por_usuario']].copy()
df_control = df_control.rename(columns={'ticket_medio_por_usuario': 'ticket_medio_controle'})

# Junta os dados pelo segmento de uso
df_roi = df_target.merge(df_control, on='categoria_uso', how='left')

# Calcula receita incremental por usuário dentro do segmento
df_roi['receita_incremental_por_usuario'] = df_roi['ticket_medio_por_usuario'] - df_roi['ticket_medio_controle']

# Receita total incremental = receita incremental por usuário × número de usuários target
df_roi['receita_total_incremental'] = df_roi['receita_incremental_por_usuario'] * df_roi['quantidade_total_usuarios']

# Custo da campanha = R$ 10 × número de usuários target
df_roi['custo_total_campanha'] = 10 * df_roi['quantidade_total_usuarios']

# ROI = (receita - custo) / custo
df_roi['roi'] = (df_roi['receita_total_incremental'] - df_roi['custo_total_campanha']) / df_roi['custo_total_campanha']

# ROI formatado como percentual legível
df_roi['roi_formatado'] = df_roi['roi'].apply(lambda x: f"{x:+.1%}")

# Exibe resultado final
display(df_roi[['categoria_uso', 'quantidade_total_usuarios', 'ticket_medio_por_usuario',
                'ticket_medio_controle', 'receita_total_incremental',
                'custo_total_campanha', 'roi_formatado']])


Apesar do ROI total positivo da campanha, ao segmentar os usuários por nível de engajamento, observamos que **em nenhuma faixa o cupom gerou retorno financeiro suficiente para cobrir seu custo**. Isso sugere que o ganho agregado da campanha se deveu ao aumento no volume de usuários que realizaram pedidos, e não ao aumento no ticket médio ou no comportamento de compra individual.

### Ticket médio por compra do usuário

In [0]:
query_segmentacao_ticket_medio = """
WITH pedidos_por_usuario AS (
    SELECT
        p.customer_id,
        ab.is_target,
        COUNT(p.order_id) AS quantidade_pedidos,
        SUM(p.order_total_amount) AS valor_total_pedidos,
        ROUND(SUM(p.order_total_amount) / COUNT(p.order_id), 2) AS ticket_medio_por_compra
    FROM
        vw_pedidos AS p
    JOIN
        vw_usuarios_teste_ab AS ab ON
            p.customer_id = ab.customer_id
    GROUP BY
        p.customer_id,
        ab.is_target
),

usuarios_com_tercil AS (
    SELECT
        *,
        NTILE(3) OVER (ORDER BY ticket_medio_por_compra) AS tercil_ticket
    FROM
        pedidos_por_usuario
),

usuarios_com_categoria AS (
    SELECT
        *,
        CASE
            WHEN tercil_ticket = 1 THEN '1) baixo ticket'
            WHEN tercil_ticket = 2 THEN '2) ticket médio'
            ELSE '3) alto ticket'
        END AS categoria_ticket
    FROM
        usuarios_com_tercil
),

agregado_por_categoria AS (
    SELECT
        categoria_ticket,
        is_target,
        COUNT(*) AS quantidade_total_usuarios,
        SUM(quantidade_pedidos) AS quantidade_total_pedidos,
        ROUND(SUM(quantidade_pedidos) * 1.0 / COUNT(*), 2) AS pedidos_medios_por_usuario,
        ROUND(SUM(valor_total_pedidos) / COUNT(*), 2) AS ticket_medio_por_usuario,
        ROUND(SUM(valor_total_pedidos) / SUM(quantidade_pedidos), 2) AS ticket_medio_por_pedido
    FROM
        usuarios_com_categoria
    GROUP BY
        categoria_ticket,
        is_target
)

SELECT
    *
FROM
    agregado_por_categoria
ORDER BY
    categoria_ticket,
    is_target
"""

df_segmentacao_ticket_medio = spark.sql(query_segmentacao_ticket_medio)

display(df_segmentacao_ticket_medio)

In [0]:
# Converte o DataFrame completo da segmentação para Pandas
df_uso = df_segmentacao_ticket_medio.toPandas()

# Separa os dados de target e controle
df_target = df_uso[df_uso['is_target'] == 'target'].copy()
df_control = df_uso[df_uso['is_target'] == 'control'][['categoria_ticket', 'ticket_medio_por_usuario']].copy()
df_control = df_control.rename(columns={'ticket_medio_por_usuario': 'ticket_medio_controle'})

# Junta os dados pelo segmento de ticket
df_roi = df_target.merge(df_control, on='categoria_ticket', how='left')

# Calcula receita incremental por usuário
df_roi['receita_incremental_por_usuario'] = df_roi['ticket_medio_por_usuario'] - df_roi['ticket_medio_controle']

# Receita total incremental
df_roi['receita_total_incremental'] = df_roi['receita_incremental_por_usuario'] * df_roi['quantidade_total_usuarios']

# Custo da campanha (R$ 10 por usuário)
df_roi['custo_total_campanha'] = 10 * df_roi['quantidade_total_usuarios']

# ROI bruto
df_roi['roi'] = (df_roi['receita_total_incremental'] - df_roi['custo_total_campanha']) / df_roi['custo_total_campanha']

# ROI formatado
df_roi['roi_formatado'] = df_roi['roi'].apply(lambda x: f"{x:+.1%}")

# Exibe resultado final
display(df_roi[['categoria_ticket', 'quantidade_total_usuarios', 'ticket_medio_por_usuario',
                'ticket_medio_controle', 'receita_total_incremental',
                'custo_total_campanha', 'roi_formatado']])


Ao segmentar os usuários pelo ticket médio por compra, observamos que quanto maior o padrão de gasto do cliente, maior o retorno gerado pelo cupom. Isso sugere que a personalização das campanhas com base em comportamento de consumo pode aumentar significativamente a eficiência financeira das ações promocionais.

### Estado (UF) do pedido

In [0]:
query_segmentacao_uf = """
WITH pedidos_por_usuario AS (
    SELECT
        p.customer_id,
        ab.is_target,
        p.delivery_address_state AS uf,
        COUNT(p.order_id) AS quantidade_pedidos,
        SUM(p.order_total_amount) AS valor_total_pedidos
    FROM
        vw_pedidos AS p
    JOIN
        vw_usuarios_teste_ab AS ab ON
            p.customer_id = ab.customer_id
    GROUP BY
        p.customer_id,
        ab.is_target,
        p.delivery_address_state
),

agregado_por_uf AS (
    SELECT
        uf,
        is_target,
        COUNT(*) AS quantidade_total_usuarios,
        SUM(quantidade_pedidos) AS quantidade_total_pedidos,
        ROUND(SUM(quantidade_pedidos) * 1.0 / COUNT(*), 2) AS pedidos_medios_por_usuario,
        ROUND(SUM(valor_total_pedidos) / COUNT(*), 2) AS ticket_medio_por_usuario,
        ROUND(SUM(valor_total_pedidos) / SUM(quantidade_pedidos), 2) AS ticket_medio_por_pedido
    FROM
        pedidos_por_usuario
    GROUP BY
        uf,
        is_target
)

SELECT
    *
FROM
    agregado_por_uf
ORDER BY
    uf,
    is_target
"""

df_segmentacao_uf = spark.sql(query_segmentacao_uf)

display(df_segmentacao_uf)

In [0]:
# Converte o DataFrame de segmentação por UF para Pandas
df_uf = df_segmentacao_uf.toPandas()

# Separa os dados de target e controle
df_target = df_uf[df_uf['is_target'] == 'target'].copy()
df_control = df_uf[df_uf['is_target'] == 'control'][['uf', 'ticket_medio_por_usuario']].copy()
df_control = df_control.rename(columns={'ticket_medio_por_usuario': 'ticket_medio_controle'})

# Junta os dados pelo estado
df_roi = df_target.merge(df_control, on='uf', how='left')

# Receita incremental por usuário
df_roi['receita_incremental_por_usuario'] = df_roi['ticket_medio_por_usuario'] - df_roi['ticket_medio_controle']

# Receita total incremental
df_roi['receita_total_incremental'] = df_roi['receita_incremental_por_usuario'] * df_roi['quantidade_total_usuarios']

# Custo da campanha = R$ 10 por usuário target
df_roi['custo_total_campanha'] = 10 * df_roi['quantidade_total_usuarios']

# ROI bruto
df_roi['roi'] = (df_roi['receita_total_incremental'] - df_roi['custo_total_campanha']) / df_roi['custo_total_campanha']

# ROI formatado
df_roi['roi_formatado'] = df_roi['roi'].apply(lambda x: f"{x:+.1%}")

# Exibe resultado final
display(df_roi[['uf', 'quantidade_total_usuarios', 'ticket_medio_por_usuario',
                'ticket_medio_controle', 'receita_total_incremental',
                'custo_total_campanha', 'roi_formatado']])


A análise por estado mostra que a campanha foi altamente rentável nos maiores centros urbanos, com ROI superior a 150% em SP, RJ e DF. Estados do Nordeste também responderam bem. Em contrapartida, regiões como SC, MA e PI apresentaram ROI negativo, indicando a necessidade de ajustes regionais na estratégia de incentivos.

### Plataforma onde foi feito o pedido

In [0]:
query_segmentacao_plataforma = """
WITH pedidos_por_usuario AS (
    SELECT
        p.customer_id,
        ab.is_target,
        p.origin_platform,
        COUNT(p.order_id) AS quantidade_pedidos,
        SUM(p.order_total_amount) AS valor_total_pedidos
    FROM
        vw_pedidos AS p
    JOIN
        vw_usuarios_teste_ab AS ab ON
            p.customer_id = ab.customer_id
    WHERE
        p.origin_platform IS NOT NULL
    GROUP BY
        p.customer_id,
        ab.is_target,
        p.origin_platform
),

agregado_por_plataforma AS (
    SELECT
        origin_platform,
        is_target,
        COUNT(*) AS quantidade_total_usuarios,
        SUM(quantidade_pedidos) AS quantidade_total_pedidos,
        ROUND(SUM(quantidade_pedidos) * 1.0 / COUNT(*), 2) AS pedidos_medios_por_usuario,
        ROUND(SUM(valor_total_pedidos) / COUNT(*), 2) AS ticket_medio_por_usuario,
        ROUND(SUM(valor_total_pedidos) / SUM(quantidade_pedidos), 2) AS ticket_medio_por_pedido
    FROM
        pedidos_por_usuario
    GROUP BY
        origin_platform,
        is_target
)

SELECT
    *
FROM
    agregado_por_plataforma
ORDER BY
    origin_platform,
    is_target
"""

df_segmentacao_plataforma = spark.sql(query_segmentacao_plataforma)

display(df_segmentacao_plataforma)

In [0]:
# Converte o DataFrame para Pandas (caso ainda não esteja)
df_uso = df_segmentacao_plataforma.toPandas()

# Separa dados de target e controle
df_target = df_uso[df_uso['is_target'] == 'target'].copy()
df_control = df_uso[df_uso['is_target'] == 'control'][['origin_platform', 'ticket_medio_por_usuario']].copy()
df_control = df_control.rename(columns={'ticket_medio_por_usuario': 'ticket_medio_controle'})

# Junta target com controle por plataforma
df_roi = df_target.merge(df_control, on='origin_platform', how='left')

# Receita incremental por usuário
df_roi['receita_incremental_por_usuario'] = df_roi['ticket_medio_por_usuario'] - df_roi['ticket_medio_controle']

# Receita total incremental
df_roi['receita_total_incremental'] = df_roi['receita_incremental_por_usuario'] * df_roi['quantidade_total_usuarios']

# Custo da campanha (R$ 10 por usuário target)
df_roi['custo_total_campanha'] = 10 * df_roi['quantidade_total_usuarios']

# ROI bruto
df_roi['roi'] = (df_roi['receita_total_incremental'] - df_roi['custo_total_campanha']) / df_roi['custo_total_campanha']

# ROI formatado
df_roi['roi_formatado'] = df_roi['roi'].apply(lambda x: f"{x:+.1%}")

# Exibe resultado final
display(df_roi[['origin_platform', 'quantidade_total_usuarios', 'ticket_medio_por_usuario',
                'ticket_medio_controle', 'receita_total_incremental',
                'custo_total_campanha', 'roi_formatado']])


A análise por plataforma de origem revelou que a campanha apresentou os melhores resultados financeiros entre os usuários de iOS e desktop, com ROIs de +166,5% e +116,0%, respectivamente. Esses canais não apenas demonstraram maior engajamento, como também ticket médio superior ao grupo controle, indicando maior propensão ao consumo com incentivo. 

Usuários de Android também responderam positivamente à campanha, embora com retorno mais modesto (+89,4%), sugerindo espaço para estratégias específicas de ativação ou diferenciação no valor do cupom. 

Já usuários de Windows Phone, além de representarem uma base pouco expressiva, tiveram ROI significativamente mais baixo (+22,8%), o que indica que esse público pode ser excluído ou receber incentivos mais restritos em ações futuras.

As segmentações realizadas permitiram identificar perfis de usuários com diferentes níveis de resposta à campanha de cupons. Usuários com maior intensidade de uso e ticket médio mais alto foram os que apresentaram maior retorno sobre o investimento, confirmando que clientes mais engajados respondem melhor a incentivos. Na análise geográfica, os grandes centros urbanos como SP, RJ e DF se destacaram, com elevado ROI e grande base de usuários, evidenciando maior maturidade e aderência ao canal. Por fim, a segmentação por plataforma mostrou que iOS e desktop são canais especialmente rentáveis, com desempenho superior ao Android e Windows Phone. Esses resultados reforçam a importância de direcionar campanhas com base em comportamento e perfil de consumo, otimizando o uso de recursos e maximizando retorno.

-----

3. Com base na análise que realizou nas questões 1 e 2, **sugira os próximos passos que o iFood deve tomar**. Lembre-se que você precisa defender suas sugestões para as lideranças de Negócio, por isso não esqueça de **incluir uma previsão de impacto** (financeiro ou não) caso o iFood siga com a sua recomendação. Fique à vontade para sugerir melhorias no processo/teste e para propor diferentes estratégias de acordo com cada segmento de usuário.

## 3. Próximos passos recomendados para o iFood

Com base nas análises realizadas, a campanha de cupons apresentou retorno financeiro positivo no agregado, com ROI de aproximadamente **+160%**, além de impulsionar a retenção e o volume de pedidos. No entanto, segmentações detalhadas revelaram **diferentes níveis de efetividade** entre os perfis de usuários.

### 1. Direcionar campanhas futuras para perfis com maior retorno
- Priorizar usuários com:
  - **Alto ticket médio**
  - **Alta frequência de pedidos**
  - **Plataforma iOS e desktop**
- Nesses grupos, o ROI superou **+140%** e chegou a **+278%** em heavy spenders.
- **Previsão de impacto**: manter o investimento nos 50% mais responsivos pode mais que dobrar o retorno da campanha sem aumento de custo.

### 2. Reduzir ou eliminar incentivos para segmentos pouco responsivos
- Usuários com **baixo uso**, **ticket médio muito baixo** e **plataformas legadas (ex: Windows Phone)** apresentaram ROI abaixo de 0%, chegando a **-100%**.
- Estados como **SC, MA e PI** também mostraram ROI negativo.
- **Previsão de impacto**: eliminar cupom para esses grupos pode gerar uma economia de até **R$ 1,5 milhão**, sem perda de receita incremental.

### 3. Testar valores de cupom personalizados
- Usuários Android responderam bem, mas com ROI inferior ao iOS.
- Recomenda-se testar **valores escalonados de cupom por segmento**, por exemplo:
  - R$ 5 para usuários de ticket baixo
  - R$ 10 para ticket médio
  - R$ 15 para heavy spenders

### 4. Criar réguas de incentivo baseadas em comportamento
- A campanha atual foi distribuída de forma uniforme.
- Próximo passo: desenvolver modelos de propensão e **aplicar regras de elegibilidade mais inteligentes**, priorizando usuários com maior probabilidade de recompra e ticket elevado.
- **Previsão de impacto**: esse ajuste pode elevar o ROI médio da campanha acima de **+200%**, mantendo o mesmo orçamento.

### Conclusão

A campanha demonstrou sucesso no agregado, mas revelou oportunidades claras de otimização por meio de segmentação. Ao focar nos perfis mais responsivos e evitar investimento em usuários de baixo retorno, o iFood pode dobrar sua eficiência promocional e transformar cupons em uma alavanca ainda mais poderosa de crescimento com sustentabilidade financeira.


## Sugestões para aprimoramento de futuros testes A/B

Com base nos resultados observados nesta campanha e nas segmentações aplicadas, seguem abaixo recomendações para o desenho de novos testes A/B e estratégias promocionais mais precisas e eficazes.

### 1. Análise por tipo de restaurante
- Incluir a variável de **segmento gastronômico** (ex: pizzarias, hamburguerias, comidas saudáveis) como critério de análise.
- Possíveis hipóteses:
  - O impacto do cupom pode variar de acordo com o tipo de restaurante.
  - Estabelecimentos com ticket médio mais alto (ex: sushi) podem gerar maior retorno com cupons.
- A segmentação por categoria também permite que o iFood ofereça **cupons personalizados por vertical**, aumentando a relevância do incentivo.

### 2. Análise pré e pós cupom (período de observação anterior)
- Avaliar o comportamento do grupo `target` **antes da distribuição dos cupons** e comparar com o período pós-uso:
  - Quantidade média de pedidos
  - Ticket médio por usuário
  - Tempo entre os pedidos
- Essa análise permite medir **mudanças reais de comportamento**, fortalecendo a avaliação causal do teste.

### 3. Testar diferentes valores e formatos de incentivo
- Avaliar o impacto de diferentes tipos de estímulo:
  - Desconto fixo (ex: R$ 10)
  - Percentual (ex: 20% off)
  - Frete grátis
  - Cashback escalonado (quanto mais pedidos, maior o retorno)
- Esses testes ajudam a entender **qual incentivo é mais eficaz para cada segmento de usuário**.

### 4. Personalização com base em comportamento histórico
- Utilizar variáveis como:
  - Frequência de compra nos últimos 3 meses
  - Ticket médio histórico
  - Categoria preferida de restaurante
- Criar **clusters de comportamento** e aplicar regras de cupom personalizadas para cada grupo.

### 5. Análise de efeito de canal de ativação
- Medir se o impacto do cupom varia conforme o canal de comunicação:
  - Push notification
  - E-mail
  - Banner in-app
- Pode-se realizar testes A/B incluindo a variável **canal de entrega do cupom** para mensurar qual canal gera maior ativação.

### 6. Medir o efeito no longo prazo (retenção estendida)
- Estender o período de observação além da campanha para entender:
  - Quantos usuários continuam ativos 30, 60 ou 90 dias após o uso do cupom.
  - Qual foi o LTV incremental por grupo.
- Permite calcular o **retorno real ao longo do ciclo de vida do cliente**.

### 7. Considerar teste A/B multivariado
- Em vez de testar apenas a presença do cupom, testar múltiplas variáveis ao mesmo tempo:
  - Tipo de incentivo × valor do cupom × canal de ativação
- Com um bom desenho experimental, é possível descobrir **interações entre fatores** que maximizam o engajamento.

### Conclusão

As sugestões acima permitem que futuros testes A/B no iFood sejam mais granulares, personalizados e orientados a valor. Ao considerar diferentes perfis de usuário, tipos de restaurante, canais e formatos de incentivo, a empresa pode **aumentar significativamente a efetividade das campanhas promocionais**, reduzindo custos e maximizando retenção e receita incremental.
